In [2]:
!pip install --upgrade pyngrok
from pyngrok import ngrok
from google.colab import userdata

ngrok.set_auth_token(userdata.get("NGROK_TOKEN"))

In [3]:
ngrok.connect(5000).public_url

'https://bbef-34-127-51-245.ngrok-free.app'

In [4]:
!pip install flask
!pip install flask-jwt-extended
!pip install keras-nlp

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 691.2/691.2 kB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 84.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 615.3/615.3 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 92.9 MB/s eta 0:00:00
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.17.1
    Uninstalling tensorboard-2.17.1:
      Successfully uninstalled tensorboard-2.17.1
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.17.1
    Uninstalling tensorflow-2.17.1:
      Successfully uninstalled tensorflow-2.17.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tf-keras 2.17.0 requires tensorflow<2.18,>=2.17, but you have tensorflow 2.18.0 which is incompatible.


In [5]:
import os

from google.colab import userdata

os.environ["KAGGLE_USERNAME"] = userdata.get('KAGGLE_USERNAME')

os.environ["KAGGLE_KEY"] = userdata.get('KAGGLE_KEY')

os.environ["KERAS_BACKEND"] = "jax"

# Avoid memory fragmentation on JAX backend.

os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"]= "1.00"

In [6]:
import keras

import keras_nlp

__START_TURN_USER__ = "<start_of_turn>user\n"

__START_TURN_MODEL__ = "<start_of_turn>model\n"

__END_TURN__ = "<end_of_turn>\n"

In [7]:
from flask import Flask, request, jsonify

from flask_jwt_extended import JWTManager, create_access_token, jwt_required, get_jwt_identity

app = Flask(__name__)

# Konfigurasi JWT

app.config["JWT_SECRET_KEY"] = os.getenv("JWT_SECRET_KEY", "default_jwt_secret") # Ganti dengan key rahasia
jwt = JWTManager(app)
print("Loading Model")
llm_model = keras_nlp.models.GemmaCausalLM.from_preset("gemma_1.1_instruct_2b_en")
print("Ready to serve…")
user_histories = {}

Loading Model


100%|██████████| 554/554 [00:00<00:00, 480kB/s]


100%|██████████| 4.67G/4.67G [01:34<00:00, 52.8MB/s]


100%|██████████| 401/401 [00:00<00:00, 376kB/s]


100%|██████████| 4.04M/4.04M [00:00<00:00, 7.74MB/s]


Ready to serve…


In [8]:
@app.route("/login", methods=["POST"])

def login():
    if request.method == 'POST':
        username = request.form.get("user")
        password = request.form.get("password")
        print(username)
        print(password)

        # Validasi user (hardcoded, gunakan database di produksi)
        if username in ["user01","user02"] and password == "password":
            access_token = create_access_token(identity=username)
            return jsonify({"access_token": access_token}), 200
        else:
            return jsonify({"error": "Invalid credentials"}), 401
    return jsonify({"error": "Invalid Request"}), 401

In [9]:
@app.route("/chat_gemma", methods=["POST"])
@jwt_required()
def chat():
    current_user = get_jwt_identity()
    print(current_user)
    user_input = request.form.get("input_text", "")
    if not user_input:
        return jsonify({"error": "No input provided"}), 400
    print(user_input)
    if current_user not in user_histories:
        user_histories[current_user] = []

    user_histories[current_user].append(f"user: {user_input}")
    user_histories[current_user].append(f"model: {response}")

    return jsonify({
        "response": "",
        "history": user_histories[current_user]
    })

In [10]:
@app.route("/history", methods=["GET"])
@jwt_required()
def get_history():
    # Identifikasi pengguna
    current_user = get_jwt_identity()

    # Ambil riwayat dari penyimpanan di memori
    history = user_histories.get(current_user, "")

    return jsonify({"user": current_user, "history": history})

if __name__ == "__main__":
    app.run(port=5000)

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [08/Dec/2024 11:19:02] "POST /?user=user01 HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [08/Dec/2024 11:19:22] "POST /?user=user01 HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [08/Dec/2024 11:20:01] "POST /?user=user01&password HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [08/Dec/2024 11:21:05] "POST /?username=user01&password HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [08/Dec/2024 11:21:19] "POST /?username=user01&password=password HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [08/Dec/2024 11:21:22] "POST /?user=user01&password=password HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [08/Dec/2024 11:21:27] "POST /?user=user01&password=password HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [08/Dec/2024 11:24:11] "POST / HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [08/